In [ ]:
"""import tensorflow as tf
tf.test.gpu_device_name()
!nvidia-smi"""
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']


In [ ]:
#Imports
import pandas as pd
import numpy as np
import nltk
%pip install --upgrade gensim
%pip install --upgrade keras
from gensim import *
%pip install autocorrect
from autocorrect import Speller
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
from google.colab import files
import io
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, Conv1D, MaxPooling1D, Dropout, LSTM
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import *
from sklearn.model_selection import StratifiedKFold

#Defines
spell = Speller(lang = 'en')
stopWordsSet = set(nltk.corpus.stopwords.words('english'))
results = {}

Requirement already up-to-date: gensim in /usr/local/lib/python3.7/dist-packages (4.0.1)
Requirement already up-to-date: keras in /usr/local/lib/python3.7/dist-packages (2.4.3)
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
#data set preprocessing
def preprocess(data,colname,applyList=[1,2,3]):
    def docPreprocess(badText):
        funcDict = {}
        funcDict[1]=lambda x:[spell(word) for word in x]
        funcDict[2]=lambda x:[nltk.stem.SnowballStemmer('english').stem(word) for word in x]
        funcDict[3]=lambda x:[nltk.stem.WordNetLemmatizer().lemmatize(word) for word in betterText]
        betterText = badText.split()
        betterText = [word for word in betterText if word not in stopWordsSet]
        betterText = [word for word in betterText if word not in freq]
        for i in applyList:
            betterText = funcDict[i](betterText)
        return betterText
    stopWordsSet = set(nltk.corpus.stopwords.words('english'))
    spell = Speller(lang = 'en')
    #
    data = data.copy()
    data[colname] = data[colname].str.replace("[0-9]", "").str.replace("[^\s\w]", "").str.lower().str.encode('ascii', 'ignore').str.decode('ascii')
    freq = pd.Series(' '.join(data[colname]).split()).value_counts()[:10]
    data[colname] = data[colname].apply(docPreprocess)
    data = data.dropna()
    return data

def short_preprocess(data,colname):
  data[colname] = data[colname].str.replace("[0-9]", "").str.replace("[^\s\w]", "").str.lower().str.encode('ascii', 'ignore').str.decode('ascii').apply(lambda x:x.split()).apply(lambda x:None if not x else x)
  data = data.dropna(axis=0)
  return data

In [ ]:
def tokenize(data):
  return data.apply(lambda row: nltk.word_tokenize(row['sentence']), axis=1)

In [ ]:
#import
trainFile = pd.read_csv("https://raw.githubusercontent.com/Danne95/deepLearning/main/train.csv")
testFile = pd.read_csv("https://raw.githubusercontent.com/Danne95/deepLearning/main/test.csv")

#preprocessing
train = short_preprocess(trainFile,'sentence')
test = short_preprocess(testFile,'sentence')

In [ ]:
#word2vec model
trainWV = models.Word2Vec(train['sentence'].append(test['sentence']),vector_size=300,sg=1,min_count=1,epochs=30)
testWV = models.Word2Vec(test,vector_size=300,sg=1,min_count=1,epochs=30)

trainSplit, testSplit = train_test_split(train, test_size=0.3)

#word count=
wc = max(train['sentence'].append(test['sentence']).apply(len))

#average vector function
def Matrix(row,wv):
  retlist = np.zeros((wc, wv.vector_size)).astype('float32')
  for i in range(len(row)):
    retlist[i] = wv[row[i]]
  return retlist

def rowMatrix(data,wv):
  retlist = []
  for row in data:
    if len(row)>0:
      retlist.append(Matrix(row,wv))
    else:
      data = data.drop(index)
  return np.array(retlist)

In [ ]:
trainSplitMatrix = rowMatrix(trainSplit['sentence'],trainWV.wv)
testSplitMatrix = rowMatrix(testSplit['sentence'],trainWV.wv)
testMatrix = rowMatrix(test['sentence'],trainWV.wv)

In [ ]:
model = Sequential()
model.add(LSTM(128, return_sequences=True,input_shape=(wc,trainWV.wv.vector_size)))
model.add(Dropout(0.9))
model.add(LSTM(64, return_sequences=True))
model.add(Dropout(0.9))
model.add(LSTM(16, return_sequences=True))
model.add(Flatten())
model.add(Dense(1, activation="sigmoid"))
model.summary()
# compile the keras model
model.compile(optimizer="adam",loss="binary_crossentropy", metrics=["accuracy"])
# fit the keras model on the dataset
model.fit(trainSplitMatrix,trainSplit['label'], epochs=100)
# evaluate the keras model
something, accuracy = model.evaluate(testSplitMatrix,testSplit['label'])
predictions = (model.predict(testMatrix) > 0.5).astype("int32")#model.predict_classes(testMatrix)
print('Accuracy: %.2f' % (accuracy*100))
#print(predictions)

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_15 (LSTM)               (None, 191, 128)          219648    
_________________________________________________________________
dropout_10 (Dropout)         (None, 191, 128)          0         
_________________________________________________________________
lstm_16 (LSTM)               (None, 191, 64)           49408     
_________________________________________________________________
dropout_11 (Dropout)         (None, 191, 64)           0         
_________________________________________________________________
lstm_17 (LSTM)               (None, 191, 16)           5184      
_________________________________________________________________
flatten_5 (Flatten)          (None, 3056)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                

In [ ]:
df = pd.DataFrame({'id': [i+3300 for i in range(len(predictions))],'label': [item[0] for item in predictions]})
df.to_csv("kagle_pred.csv",index=False)